In [ ]:
from replenishment import (
    build_percentile_forecast_candidates_from_standard_rows,
    PercentileForecastOptimizationPolicy,
    build_point_forecast_article_configs_from_standard_rows,
    build_replenishment_decisions_from_simulations,
    generate_standard_simulation_rows,
    optimize_aggregation_windows,
    optimize_forecast_targets,
    optimize_service_level_factors,
    replenishment_decision_rows_to_dataframe,
    split_standard_simulation_rows,
    simulate_replenishment,
    simulate_replenishment_with_aggregation,
    standard_simulation_rows_to_dataframe,
)


In [ ]:
rows = generate_standard_simulation_rows(
    n_unique_ids=3,
    periods=8,
    forecast_start_period=6,
    history_mean=18,
    history_std=4,
    forecast_mean=20,
    forecast_std=3,
    seed=7,
)
df = standard_simulation_rows_to_dataframe(rows, library="pandas")
df.head()


In [ ]:
backtest_rows, forecast_rows = split_standard_simulation_rows(rows)
point_configs = build_point_forecast_article_configs_from_standard_rows(
    backtest_rows,
    service_level_factor=0.9,
)
point_result = optimize_service_level_factors(
    point_configs,
    candidate_factors=[0.8, 0.9, 0.95],
)
point_result


In [ ]:
aggregation_result = optimize_aggregation_windows(
    point_configs,
    candidate_windows=[1, 2, 3],
)
best_factors = {
    unique_id: result.service_level_factor
    for unique_id, result in point_result.items()
}
forecast_point_configs = build_point_forecast_article_configs_from_standard_rows(
    forecast_rows,
    service_level_factor=best_factors,
)
forecast_simulations = {
    unique_id: simulate_replenishment_with_aggregation(
        periods=config.periods,
        demand=config.demand,
        initial_on_hand=config.initial_on_hand,
        lead_time=config.lead_time,
        policy=config.policy,
        aggregation_window=aggregation_result[unique_id].window,
        holding_cost_per_unit=config.holding_cost_per_unit,
        stockout_cost_per_unit=config.stockout_cost_per_unit,
        order_cost_per_order=config.order_cost_per_order,
        order_cost_per_unit=config.order_cost_per_unit,
    )
    for unique_id, config in forecast_point_configs.items()
}
decision_rows = build_replenishment_decisions_from_simulations(
    forecast_rows,
    forecast_simulations,
    aggregation_window={
        unique_id: aggregation_result[unique_id].window
        for unique_id in forecast_point_configs
    },
)
replenishment_decision_rows_to_dataframe(decision_rows, library="pandas")


In [ ]:
percentile_configs = build_percentile_forecast_candidates_from_standard_rows(backtest_rows)
percentile_result = optimize_forecast_targets(
    percentile_configs,
)
percentile_result


In [ ]:
forecast_percentile_configs = build_percentile_forecast_candidates_from_standard_rows(
    forecast_rows,
)
forecast_percentile_simulations = {}
for unique_id, config in forecast_percentile_configs.items():
    target = percentile_result[unique_id].target
    policy = PercentileForecastOptimizationPolicy(
        forecast=config.forecast_candidates[target],
        lead_time=config.lead_time,
    )
    forecast_percentile_simulations[unique_id] = simulate_replenishment(
        periods=config.periods,
        demand=config.demand,
        initial_on_hand=config.initial_on_hand,
        lead_time=config.lead_time,
        policy=policy,
        holding_cost_per_unit=config.holding_cost_per_unit,
        stockout_cost_per_unit=config.stockout_cost_per_unit,
        order_cost_per_order=config.order_cost_per_order,
        order_cost_per_unit=config.order_cost_per_unit,
    )

percentile_decision_rows = build_replenishment_decisions_from_simulations(
    forecast_rows,
    forecast_percentile_simulations,
)
replenishment_decision_rows_to_dataframe(percentile_decision_rows, library="pandas")
